## Imports and Setup

In [1]:
import os
import pandas as pd
import geopandas as gpd
import geemap.foliumap as emap

import sys
sys.path.insert(0, "../utils/")
import gee_utils
import config_utils
import clean_utils

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

%load_ext autoreload
%autoreload 2

## Data Configs

In [2]:
cwd = os.path.dirname(os.getcwd())
config_file = os.path.join(cwd, 'configs/data_config.yaml')
config = config_utils.create_config(config_file)

## Option 1: Download Directly from Data Source (Recommended)
Download the GHS-BUILT-C Product directly from the JRC website: https://ghsl.jrc.ec.europa.eu/download.php?ds=builtC

In [3]:
ghsl_zip = os.path.join(cwd, config["rasters_dir"], "ghsl/global/ghsl.zip")
ghsl_folder = os.path.join(cwd, config["rasters_dir"], "ghsl/global")
ghsl_path = os.path.join(cwd, config["rasters_dir"], 'ghsl', config["ghsl_file"])

if not os.path.exists(ghsl_path):
    !wget {config["ghsl_url"]} -O {ghsl_zip}
    !unzip {ghsl_zip} -d {ghsl_folder}
    
clean_utils.generate_ghsl_per_country(config, layer="ghsl", exclude=["KIR"])

Processing HUN: 100%|████████████████████| 41/41 [00:00<00:00, 1854.97it/s]                                             


## Option 2: Download from Google Earth Engine (GEE)
Download the GHSL Product via Google Earth Engine. Cropping the GHSL dataset per country may cause the system to crash; downloading from GEE is a good alternative for getting country-level GHSL data.

### Export GEE Images to Google Drive

In [4]:
gee_utils._initialize_gee()
iso_codes = ["KIR"]
gee_utils.generate_gee_images(config, iso_codes=iso_codes, folder='GHSL', layer="ghsl")

Enter verification code:  4/1AfJohXlyDSjzOabd7dUIAtur_WpGHDq6YnPmoOwjiY-MgSpyfUTvUVepPVI



Successfully saved authorization token.


Processing KIR: 100%|████████████████████| 1/1 [00:06<00:00,  6.08s/it]                                                 


### Merge and Upload GeoTiffs

The `generate_gee_images()` function will export the GEE images to your GDrive folder. Download these images to your local system. Some TIFFs maybe broken down into multiple files; to merge these, simply use the following command:
```sh
conda install gdal
gdalbuildvrt ISO_ghsl.vrt ./*.tif
gdal_translate -of GTiff -co "TILED=YES" ISO_ghsl.vrt ISO_ghsl.tif
```
Once merged, upload the GeoTIFF place it in your GHSL rasters folder under the file name `{iso_code}_ghsl.tiff`, e.g. `data/rasters/ghsl/NAM_ghsl.tiff`. 